# Assignment 2

In this assignment you will create a coordinate-based multilayer perceptron in numpy from scratch. For each input image coordinate $(x,y)$, the model predicts the associated color $(r,g,b)$.

![Network diagram](https://user-images.githubusercontent.com/3310961/85066930-ad444580-b164-11ea-9cc0-17494679e71f.png)

You will then compare the following input feature mappings $\gamma (\mathbf{v})$.

- No mapping: $\gamma(\mathbf{v})= \mathbf{v}$. 

- Basic mapping: $\gamma(\mathbf{v})=\left[ \cos(2 \pi \mathbf{v}),\sin(2 \pi \mathbf{v}) \right]^\mathrm{T}$. 

- Gaussian Fourier feature mapping: $\gamma(\mathbf{v})= \left[ \cos(2 \pi \mathbf B \mathbf{v}), \sin(2 \pi \mathbf B \mathbf{v}) \right]^\mathrm{T}$, 
where each entry in $\mathbf B \in \mathbb R^{m \times d}$ is sampled from $\mathcal N(0,\sigma^2)$.

Some notes to help you with that:

- You will implement the mappings in the helper functions `get_B_dict` and `input_mapping`. 
- The basic mapping can be considered a case where $\mathbf B \in \mathbb R^{2 \times 2}$ is the indentity matrix. 
- For this assignment, $d$ is 2 because the input coordinates in two dimensions. 
- You can experiment with $m$ and $\sigma$ values e.g. $m=256$ and $\sigma \in \{1, 10, 100\}$.

Source: https://bmild.github.io/fourfeat/ 
This assignment is inspired by and built off of the authors' demo. 

## Setup

### Imports

In [ ]:
import matplotlib.pyplot as plt
import os, imageio
import cv2
import numpy as np

# imports /content/assignment2/models/neural_net.py if you mounted correctly 
from models.neural_net import NeuralNetwork

# makes sure your NeuralNetwork updates as you make changes to the .py file
%load_ext autoreload
%autoreload 2

# sets default size of plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  

## Helper Functions

### Experiment Runner (Fill in TODOs)

In [ ]:
def NN_experiment(X_train, y_train, X_test, y_test, input_size, num_layers,\
                  hidden_size, hidden_sizes, output_size, epochs,\
                  learning_rate, opt, batch_size, weight_decay, beta1):
  
    # Initialize a new neural network model
    net = NeuralNetwork(input_size, hidden_sizes, output_size, num_layers, opt, batch_size=batch_size)

    # Variables to store performance for each epoch
    train_loss = np.zeros(epochs)
    train_psnr = np.zeros(epochs)
    test_psnr = np.zeros(epochs)
    test_loss = np.zeros(epochs)
    predicted_images = np.zeros((epochs, y_test.shape[0], y_test.shape[1]))

    BATCH_SIZE = batch_size

    cur_lr = learning_rate
    step = 0
    for epoch in (range(epochs)):
        
        
        # Shuffle the dataset
        # TODO implement this
        index = np.random.permutation(X_train.shape[0])
        X_train_shuffled = X_train[index]
        y_train_shuffled = y_train[index]

        num_step = X_train.shape[0] // BATCH_SIZE 

        loss_ = 0.0
        psnr_ = 0.0
        for i in range(num_step):
            step += 1
            # Training
            # Run the forward pass of the model to get a prediction and record the psnr
            # TODO implement this
            output = net.forward(X_train_shuffled[i*BATCH_SIZE : (i+1)*BATCH_SIZE])
            psnr_ += psnr(y_train_shuffled[i*BATCH_SIZE : (i+1)*BATCH_SIZE], output)

            # Run the backward pass of the model to compute the loss, record the loss, and update the weights
            loss_ += net.backward(y_train_shuffled[i*BATCH_SIZE : (i+1)*BATCH_SIZE])
            # print('loss:', loss_)
            net.update(t=step, lr=cur_lr, b1=beta1, weight_decay=weight_decay)

        train_loss[epoch] = loss_ / num_step
        train_psnr[epoch] = psnr_ / num_step

        if (epoch+1)%50 == 0 and epoch > 1000 and opt == 'SGD':
            cur_lr *= 0.5
        elif (epoch+1)%200 == 0 and epoch > 600 and opt == 'Adam':
            cur_lr *= 0.8

        
        # Testing
        # No need to run the backward pass here, just run the forward pass to compute and record the psnr
        predicted_images[epoch] = net.forward(X_test)
        test_psnr[epoch] = psnr(y_test, predicted_images[epoch])
        test_loss[epoch] = net.mse(y_test, predicted_images[epoch])

        # print('testing psnr of epoch {}: {}'.format((epoch+1), psnr(y_test[i*BATCH_SIZE : (i+1)*BATCH_SIZE], output)))
        
    return net, train_psnr, test_psnr, train_loss, predicted_images, test_loss

### Image Data and Feature Mappings (Fill in TODOs)

In [ ]:
# Data loader - already done for you
def get_image(size=512, \
              image_url='https://bmild.github.io/fourfeat/img/lion_orig.png'):

  # Download image, take a square crop from the center  
  img = imageio.imread(image_url)[..., :3] / 255.
  c = [img.shape[0]//2, img.shape[1]//2]
  r = 256
  img = img[c[0]-r:c[0]+r, c[1]-r:c[1]+r]

  if size != 512:
    img = cv2.resize(img, (size, size))

  plt.imshow(img)
  plt.show()

  # Create input pixel coordinates in the unit square
  coords = np.linspace(0, 1, img.shape[0], endpoint=False)
  x_test = np.stack(np.meshgrid(coords, coords), -1)
  test_data = [x_test, img]
  train_data = [x_test[::2, ::2], img[::2, ::2]]
  
  return train_data, test_data

In [ ]:
# Create the mappings dictionary of matrix B -  you will implement this
def get_B_dict(var_list):
  B_dict = {}
  B_dict['none'] = None
  
  # add B matrix for basic, gauss_1.0, gauss_10.0, gauss_100.0
  # TODO implement this
  B_dict['basic'] = np.eye(2)
  for i in var_list:
    B_dict['gauss_{}'.format(i)] = np.random.normal(0, i, (256, 2))

  return B_dict

In [ ]:
# Given tensor x of input coordinates, map it using B - you will implement
def input_mapping(x, B):
  if B is None:
    # "none" mapping - just returns the original input coordinates
    return x
  else:
    # "basic" mapping and "gauss_X" mappings project input features using B
    proj = 2 * np.pi * np.matmul(x, B.T)
    return np.concatenate([np.sin(proj), np.cos(proj)], axis=-1)

In [ ]:
# Apply the input feature mapping to the train and test data - already done for you
def get_input_features(B_dict, mapping):
  # mapping is the key to the B_dict, which has the value of B
  # B is then used with the function `input_mapping` to map x  
  
  y_train = train_data[1].reshape(-1, output_size)
  y_test = test_data[1].reshape(-1, output_size)
  X_train = input_mapping(train_data[0].reshape(-1, 2), B_dict[mapping])
  X_test = input_mapping(test_data[0].reshape(-1, 2), B_dict[mapping])
  return X_train, y_train, X_test, y_test


### MSE Loss and PSNR Error (Fill in TODOs)

In [ ]:
def mse(y, p):
  # TODO implement this
  # make sure it is consistent with your implementation in neural_net.py
  return np.mean((y-p)**2)

def psnr(y, p):
  # TODO implement this
  return -10 * np.log10(mse(y, p))

### Plotting

In [ ]:
def plot_training_curves(train_loss, test_loss, train_psnr, test_psnr):

  # plot the training loss
  plt.subplot(2, 1, 1)
  plt.plot(train_loss, label='train')
  plt.plot(test_loss, label='test')
  plt.title('MSE history')
  plt.xlabel('Iteration')
  plt.ylabel('MSE Loss')
  plt.legend()

  # plot the training and testing psnr
  plt.subplot(2, 1, 2)
  plt.plot(train_psnr, label='train')
  plt.plot(test_psnr, label='test')
  plt.title('PSNR history')
  plt.xlabel('Iteration')
  plt.ylabel('PSNR')
  plt.legend()

  plt.tight_layout()
  plt.show()

In [ ]:
def plot_reconstruction(p, y_test):
  p_im = p.reshape(size,size,3)
  y_im = y_test.reshape(size,size,3)

  plt.figure(figsize=(12,6))

  # plot the reconstruction of the image
  plt.subplot(1,2,1), plt.imshow(p_im), plt.title("reconstruction")

  # plot the ground truth image
  plt.subplot(1,2,2), plt.imshow(y_im), plt.title("ground truth")

  print("Final Test MSE", mse(y_test, p))
  print("Final Test psnr",psnr(y_test, p))

In [ ]:
def plot_reconstruction_progress(predicted_images, y_test, N=8):
  total = len(predicted_images)
  step = total // N
  plt.figure(figsize=(24, 4))

  # plot the progress of reconstructions
  for i, j in enumerate(range(0,total, step)):
      plt.subplot(1, N, i+1)
      plt.imshow(predicted_images[j].reshape(size,size,3))
      plt.axis("off")
      plt.title(f"iter {j}")

  # plot ground truth image
  plt.subplot(1, N+1, N+1)
  plt.imshow(y_test.reshape(size,size,3))
  plt.title('GT')
  plt.axis("off")
  plt.show()

In [ ]:
def plot_feature_mapping_comparison(outputs, gt):
  # plot reconstruction images for each mapping
  plt.figure(figsize=(24, 4))
  N = len(outputs)
  for i, k in enumerate(outputs):
      plt.subplot(1, N+1, i+1)
      plt.imshow(outputs[k]['pred_imgs'][-1].reshape(size, size, -1))
      plt.title(k)
  plt.subplot(1, N+1, N+1)
  plt.imshow(gt)
  plt.title('GT')
  plt.show()

  # plot train/test error curves for each mapping
  iters = len(outputs[k]['train_psnrs'])
  plt.figure(figsize=(16, 6))
  plt.subplot(121)
  for i, k in enumerate(outputs):
      plt.plot(range(iters), outputs[k]['train_psnrs'], label=k)
  plt.title('Train error')
  plt.ylabel('PSNR')
  plt.xlabel('Training iter')
  plt.legend()
  plt.subplot(122)
  for i, k in enumerate(outputs):
      plt.plot(range(iters), outputs[k]['test_psnrs'], label=k)
  plt.title('Test error')
  plt.ylabel('PSNR')
  plt.xlabel('Training iter')
  plt.legend()
  plt.show()

In [ ]:
# Save out video
def create_and_visualize_video(outputs, size=size, epochs=epochs, filename='training_convergence.mp4'):
  all_preds = np.concatenate([outputs[n]['pred_imgs'].reshape(epochs,size,size,3)[::25] for n in outputs], axis=-2)
  data8 = (255*np.clip(all_preds, 0, 1)).astype(np.uint8)
  f = os.path.join(filename)
  imageio.mimwrite(f, data8, fps=20)

  # Display video inline
  from IPython.display import HTML
  from base64 import b64encode
  mp4 = open(f, 'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

  N = len(outputs)
  if N == 1:
    return HTML(f'''
    <video width=256 controls autoplay loop>
          <source src="{data_url}" type="video/mp4">
    </video>
    ''')
  else:
    return HTML(f'''
    <video width=1000 controls autoplay loop>
          <source src="{data_url}" type="video/mp4">
    </video>
    <table width="1000" cellspacing="0" cellpadding="0">
      <tr>{''.join(N*[f'<td width="{1000//len(outputs)}"></td>'])}</tr>
      <tr>{''.join(N*['<td style="text-align:center">{}</td>'])}</tr>
    </table>
    '''.format(*list(outputs.keys())))

# Low Resolution Reconstruction

In [ ]:
size = 32
train_data, test_data = get_image(size)

Some suggested hyperparameter choices to help you start
- hidden layer count: 4
- hidden layer size: 256
- number of epochs: 1000
- learning reate: 1e-4


#### Low Resolution Reconstruction - SGD - None Mapping

In [ ]:
# get input features
B_dict = get_B_dict([])
mapping = 'none'
output_size = 3
X_train, y_train, X_test, y_test = get_input_features(B_dict, mapping)
batch_size = 128

# run NN experiment on input features
net, train_psnr, test_psnr, train_loss, predicted_images = NN_experiment(
    X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
    input_size=X_train.shape[1], num_layers=6, hidden_size=None, hidden_sizes=[256, 1024, 1024, 512, 256], output_size=output_size, 
    epochs=10, learning_rate=1e-2, opt='SGD', batch_size=batch_size, weight_decay=0, beta1=0.5
)

# plot results of experiment
plot_training_curves(train_loss, train_psnr, test_psnr)
plot_reconstruction(net.forward(X_test), y_test)

#### Low Resolution Reconstruction - Optimizer of your Choice - Various Input Mapping Stategies

In [ ]:
def train_wrapper(mapping, size, lr, opt, weight_decay, beta1):
  # makes it easy to run all your mapping experiments in a for loop
  # this will similar to what you did previously in the last two sections
  
  output_size = 3
  X_train, y_train, X_test, y_test = get_input_features(B_dict, mapping)
  batch_size = 64

  # run NN experiment on input features
  net, train_psnrs, test_psnrs, train_loss, predicted_images, test_loss = NN_experiment(
    X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
    input_size=X_train.shape[1], num_layers=7, hidden_size=None, hidden_sizes=[256, 256, 512, 512, 256, 256], output_size=output_size, 
    epochs=8*250, learning_rate=lr, opt=opt, batch_size=batch_size, weight_decay=weight_decay, beta1=beta1
  )

  # plot results of experiment
  plot_training_curves(train_loss, test_loss, train_psnr, test_psnr)
  plot_reconstruction(net.forward(X_test), y_test)
  plot_reconstruction_progress(predicted_images, y_test)

  return {
    'net': net, 
    'train_psnrs': train_psnrs, 
    'test_psnrs': test_psnrs,
    'train_loss': train_loss,
    'pred_imgs': predicted_images
  }



In [ ]:
outputs = {}
B_dict = get_B_dict([1.5, 2, 2.5, 5, 10]) 
for k in (B_dict):
  print("training", k)
  outputs[k] = train_wrapper(k, size, 1e-2, 'SGD', 1e-5, 0.5)

In [ ]:
# if you did everything correctly so far, this should output a nice figure you can use in your report
plot_feature_mapping_comparison(outputs, y_test.reshape(size,size,3))

# High Resolution Reconstruction

#### High Resolution Reconstruction - Optimizer of your Choice - Various Input Mapping Stategies

Repeat the previous experiment, but at the higher resolution. The reason why we have you first experiment with the lower resolution since it is faster to train and debug. Additionally, you will see how the mapping strategies perform better or worse at the two different input resolutions. 

In [ ]:
size = 128
train_data, test_data = get_image(size)

In [ ]:
outputs = {}
B_dict = get_B_dict([1, 2.5, 5, 10, 25, 50])
for k in (B_dict):
  print("training", k)
  outputs[k] = train_wrapper(k, size, 1e-3, 'Adam', 1e-4, 0.9)

#### High Resolution Reconstruction - Image of your Choice

When choosing an image select one that you think will give you interesting results or a better insight into the performance of different feature mappings and explain why in your report template. 

In [ ]:
plot_feature_mapping_comparison(outputs, y_test.reshape(32,32,3))

In [ ]:

size = 256
# TODO pick an image and replace the url string
train_data, test_data = get_image(size, image_url="https://i.redd.it/z8oprysf4i4a1.png")

In [ ]:
def NN_experiment(X_train, y_train, X_test, y_test, input_size, num_layers,\
                  hidden_size, hidden_sizes, output_size, epochs,\
                  learning_rate, opt, batch_size, weight_decay, beta1):
  
    # Initialize a new neural network model
    net = NeuralNetwork(input_size, hidden_sizes, output_size, num_layers, opt, batch_size=batch_size)

    # Variables to store performance for each epoch
    train_loss = np.zeros(epochs)
    train_psnr = np.zeros(epochs)
    test_psnr = np.zeros(epochs)
    test_loss = np.zeros(epochs)
    predicted_images = np.zeros((epochs, y_test.shape[0], y_test.shape[1]))

    # For each epoch...

    BATCH_SIZE = batch_size

    cur_lr = learning_rate
    step = 0
    for epoch in (range(epochs)):
        
        
        # Shuffle the dataset
        # TODO implement this
        index = np.random.permutation(X_train.shape[0])
        X_train_shuffled = X_train[index]
        y_train_shuffled = y_train[index]

        num_step = X_train.shape[0] // BATCH_SIZE 

        loss_ = 0.0
        psnr_ = 0.0
        for i in range(num_step):
            step += 1
            # Training
            # Run the forward pass of the model to get a prediction and record the psnr
            # TODO implement this
            output = net.forward(X_train_shuffled[i*BATCH_SIZE : (i+1)*BATCH_SIZE])
            psnr_ += psnr(y_train_shuffled[i*BATCH_SIZE : (i+1)*BATCH_SIZE], output)

            # Run the backward pass of the model to compute the loss, record the loss, and update the weights
            # TODO implement this
            loss_ += net.backward(y_train_shuffled[i*BATCH_SIZE : (i+1)*BATCH_SIZE])
            # print('loss:', loss_)
            net.update(t=step, lr=cur_lr, b1=beta1, weight_decay=weight_decay)

        train_loss[epoch] = loss_ / num_step
        train_psnr[epoch] = psnr_ / num_step

        if (epoch+1)%50 == 0 and epoch > 1000 and opt == 'SGD':
            cur_lr *= 0.5
        elif (epoch+1)%200 == 0 and epoch > 600 and opt == 'Adam':
            cur_lr *= 0.8

        
        # Testing
        # No need to run the backward pass here, just run the forward pass to compute and record the psnr
        # TODO implement this
        predicted_images[epoch] = net.forward(X_test)
        test_psnr[epoch] = psnr(y_test, predicted_images[epoch])
        test_loss[epoch] = net.mse(y_test, predicted_images[epoch])

        # print('testing psnr of epoch {}: {}'.format((epoch+1), psnr(y_test[i*BATCH_SIZE : (i+1)*BATCH_SIZE], output)))
        
    return net, train_psnr, test_psnr, train_loss, predicted_images, test_loss

In [ ]:
def train_wrapper(mapping, size, lr, opt, weight_decay, beta1):
  # TODO implement
  # makes it easy to run all your mapping experiments in a for loop
  # this will similar to what you did previously in the last two sections
  
  output_size = 3
  X_train, y_train, X_test, y_test = get_input_features(B_dict, mapping)
  batch_size = 64

  # run NN experiment on input features
  # TODO implement by using the NN_experiment() helper function
  net, train_psnrs, test_psnrs, train_loss, predicted_images, test_loss = NN_experiment(
    X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
    input_size=X_train.shape[1], num_layers=5, hidden_size=None, hidden_sizes=[128, 256, 256, 128], output_size=output_size, 
    epochs=8*150, learning_rate=lr, opt=opt, batch_size=batch_size, weight_decay=weight_decay, beta1=beta1
  )

  # plot results of experiment
  plot_training_curves(train_loss, test_loss, train_psnr, test_psnr)
  plot_reconstruction(net.forward(X_test), y_test)
  plot_reconstruction_progress(predicted_images, y_test)

  return {
    'net': net, 
    'train_psnrs': train_psnrs, 
    'test_psnrs': test_psnrs,
    'train_loss': train_loss,
    'pred_imgs': predicted_images
  }



In [ ]:
outputs = {}
B_dict = get_B_dict([1, 2.5, 5, 10, 25, 50])
for k in (B_dict):
  print("training", k)
  outputs[k] = train_wrapper(k, size, 1e-3, 'Adam', 1e-4, 0.9)

In [ ]:
plot_feature_mapping_comparison(outputs, y_test.reshape(32,32,3))

# Reconstruction Process Video (Optional)
(For Fun!) Visualize the progress of training in a video 

In [ ]:
# requires installing this additional dependency
!pip install imageio-ffmpeg

In [ ]:
# single video example
create_and_visualize_video({"gauss": {"pred_imgs": predicted_images}}, filename="training_high_res_gauss.mp4")

In [ ]:
# multi video example
create_and_visualize_video(outputs, epochs=1000, size=32)